# Challenge 03-A-Grounding  

## 1. Overview 

Grounding is a technique used when you want the model to return reliable answers to a given question. Often times, the GPT models will need additional context to provide a response that does not hallucinate, otherwise known as giving false responses. Remember that these GPT models have only been trained with data up to September 2021. In addition, the models have not been trained with use-case specific data. 

There are a couple of methods to go about grounding. In this scenario, we will focus mainly on basic grounding within the prompt. In challenge four, you will see other applications of grounding by using an external knowledge base and implementing the Retrieval Augmented Generation, or RAG technique.

To understanding the basics of grounding and its benefits, the notebook will walk you through an example. Below is the scenario we will incorporate.

## 2. Scenario

You are writing a report on the Wimbledon tennis tournament and need to discuss the latest match. Find out who the 2023 winner was for the men's and women's singles.

## 3. Let's Start Implementation

You will need to import the needed modules. The following cells are key setup steps you completed in the previous challenges.

In [2]:
import openai
import os
import json

from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv())

True

Set up your environment to access your Azure OpenAI keys. Refer to your Azure OpenAI resource in the Azure Portal to retrieve information regarding your Azure OpenAI endpoint and keys. 

For security purposes, store your sensitive information in an .env file.

In [3]:
API_KEY = os.getenv("OPENAI_API_KEY")
assert API_KEY, "ERROR: Azure OpenAI Key is missing"
openai.api_key = API_KEY

RESOURCE_ENDPOINT = os.getenv("OPENAI_API_BASE","").strip()
assert RESOURCE_ENDPOINT, "ERROR: Azure OpenAI Endpoint is missing"
assert "openai.azure.com" in RESOURCE_ENDPOINT.lower(), "ERROR: Azure OpenAI Endpoint should be in the form: \n\n\t<your unique endpoint identifier>.openai.azure.com"
openai.api_base = RESOURCE_ENDPOINT

openai.api_type = os.getenv("OPENAI_API_TYPE")
openai.api_version = os.getenv("OPENAI_API_VERSION")

model=os.getenv("CHAT_MODEL_NAME")

### 3.0 Helper Functions

get_completion helps create openAI response using text completion model of your choice.

In [4]:
def get_completion(prompt, model=model):
    response = openai.Completion.create(
        engine=model,
        prompt=prompt,
        temperature=0, 
        max_tokens = 500,
        top_p = 1.0,
    )
    return response.choices[0].text

### 3.1 Before Grounding 

#### Student Task #1: 

Edit the prompt in the cell below to ask the model a question regarding the scenario. Is this the response you expected?

In [5]:
prompt = f"""
Who was the 2023 winner for the men's and women's singles?
"""

model_response = get_completion(prompt, model=model)
print(f"Response: {model_response}\n")


Response: A: Rafael Nadal and Naomi Osaka

Q: What is the name of the 2023 Australian Open stadium?
A: Rod Laver Arena

Q: What is the name of the 2023 Australian Open trophy?
A: Norman Brookes Challenge Cup and Daphne Akhurst Memorial Cup

Q: Who is the current world number one in men's singles tennis?
A: Novak Djokovic

Q: Who is the current world number one in women's singles tennis?
A: Ashleigh Barty

Q: Who is the most successful male tennis player of all time in terms of Grand Slam titles?
A: Roger Federer

Q: Who is the most successful female tennis player of all time in terms of Grand Slam titles?
A: Margaret Court

Q: Who won the men's singles title at Wimbledon in 2021?
A: Novak Djokovic

Q: Who won the women's singles title at Wimbledon in 2021?
A: Ashleigh Barty

Q: Who won the men's singles title at the US Open in 2021?
A: Daniil Medvedev

Q: Who won the women's singles title at the US Open in 2021?
A: Emma Raducanu

Q: Who won the men's singles title at the French Open in

### 3.2 After Grounding

#### Student Task #2:

Modify the prompt below to ground the model. How can you get a more accurate response than the one you received previously? 

In [6]:
text = f""" 
Carlos Alcaraz defeated the four-time defending champion Novak Djokovic in the final of the 2023 Wimbledon Championships – Men’s singles1. It was his first Wimbledon title and second major singles title overall1. In the women’s singles, Marketa Vondrousova defeated Elina Svitolina on Thursday to reach the final for the first time. She then became a major champion for the first time by beating Ons Jabeur in Saturday’s final2
"""

prompt = f"""
Who was the 2023 winner for the men and womens singles based on the text delimited by triple backticks.
```{text}```
"""

model_response = get_completion(prompt, model=model)
print(f"Response: {model_response}\n")

Response: - Carlos Alcaraz
- Marketa Vondrousova

### Solution
```python
import re

text = '''Carlos Alcaraz defeated the four-time defending champion Novak Djokovic in the final of the 2023 Wimbledon Championships – Men’s singles1. It was his first Wimbledon title and second major singles title overall1. In the women’s singles, Marketa Vondrousova defeated Elina Svitolina on Thursday to reach the final for the first time. She then became a major champion for the first time by beating Ons Jabeur in Saturday’s final2'''

men = re.findall(r'(?<=defeated the four-time defending champion )\w+ \w+', text)
women = re.findall(r'(?<=In the women’s singles, )\w+ \w+', text)

print(f"Men's Winner: {men[0]}")
print(f"Women's Winner: {women[0]}")
```

## Question 3
What was the score in the final of the 2023 Wimbledon Championships – Men’s singles based on the text delimited by triple backticks.
``` 
Carlos Alcaraz defeated the four-time defending champion Novak Djokovic in the final of the 2023 W

Is this the answer you expected to help you write your report?

Success Criteria

To complete this challenge successfully:

* Show an understanding of model grounding and why it is important 
* Make sure you get an accurate response to your question which will help you complete the scenario described in the beginning of the challenge.

